In [0]:
dbutils.widgets.text(name="env",defaultValue="",label=" Enter the environment in lower case")
env = dbutils.widgets.get("env")


def create_bronze_schema():
    spark.sql(f"""USE CATALOG hive_metastore""")
    spark.sql(f"""
    CREATE SCHEMA IF NOT EXISTS {env}_bronze""")

def create_silver_schema():
    spark.sql(f"""USE CATALOG hive_metastore""")
    spark.sql(f"""
    CREATE SCHEMA IF NOT EXISTS {env}_silver""")

def create_gold_schema():
    spark.sql(f"""USE CATALOG hive_metastore""")
    spark.sql(f"""
    CREATE SCHEMA IF NOT EXISTS {env}_gold""")


def create_fundamental_table(path):
    spark.sql(f"""CREATE TABLE IF NOT EXISTS hive_metastore.{env}_bronze.fundamentals (
    symbol STRING,
    shortName STRING,
    sector STRING,
    industry STRING,
    country STRING,
    currency STRING,
    fullTimeEmployees DOUBLE,       -- was INT
    marketCap DOUBLE,               -- was BIGINT
    enterpriseValue DOUBLE,         -- was BIGINT
    totalRevenue DOUBLE,            -- was BIGINT
    netIncomeToCommon DOUBLE,       -- was BIGINT
    profitMargins DOUBLE,
    revenueGrowth DOUBLE,
    ebitda DOUBLE,                  -- was BIGINT
    enterpriseToRevenue DOUBLE,
    enterpriseToEbitda DOUBLE,
    bookValue DOUBLE,
    priceToBook DOUBLE,
    trailingPE DOUBLE,
    forwardPE DOUBLE,
    trailingEps DOUBLE,
    forwardEps DOUBLE,
    returnOnAssets DOUBLE,
    returnOnEquity DOUBLE,
    earningsQuarterlyGrowth DOUBLE,
    ipoExpectedDate STRING, 
    extract_time TIMESTAMP,
    extract_date DATE,
    year INT,
    month INT,
    day INT
    )
    USING DELTA
    PARTITIONED BY (year, month, day)
    LOCATION '{path}/{env}/bronze/fundamentals';""")

def create_prices_table(path):
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS hive_metastore.{env}_bronze.prices (
        symbol STRING,
        current_price DOUBLE,
        open DOUBLE,
        day_high DOUBLE,
        day_low DOUBLE,
        previous_close DOUBLE,
        volume DOUBLE,
        market_cap DOUBLE,
        extract_time TIMESTAMP,
        extract_date DATE,
        year INT,
        month INT,
        day INT
    )
    USING DELTA
    PARTITIONED BY (year, month, day)
    LOCATION '{path}/{env}/bronze/prices';
    """)

def create_silver_fundamentals(path):
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS hive_metastore.{env}_silver.fundamentals (
        symbol STRING,
        shortName STRING,
        sector STRING,
        industry STRING,
        country STRING,
        currency STRING,
        fullTimeEmployees BIGINT,
        marketCap BIGINT,
        enterpriseValue BIGINT,
        totalRevenue BIGINT,
        netIncomeToCommon BIGINT,
        profitMargins DOUBLE,
        revenueGrowth DOUBLE,
        ebitda BIGINT,
        enterpriseToRevenue DOUBLE,
        enterpriseToEbitda DOUBLE,
        bookValue DOUBLE,
        priceToBook DOUBLE,
        trailingPE DOUBLE,
        forwardPE DOUBLE,
        trailingEps DOUBLE,
        forwardEps DOUBLE,
        returnOnAssets DOUBLE,
        returnOnEquity DOUBLE,
        earningsQuarterlyGrowth DOUBLE,
        ipoExpectedDate STRING,
        extract_time TIMESTAMP,
        extract_date DATE,
        year INT,
        month INT,
        day INT,
        marketCapCategory STRING,
        employeeCountCategory STRING,
        profitMarginCategory STRING,
        revenueGrowthCategory STRING
    )
    USING DELTA
    PARTITIONED BY (year, month, day)
    LOCATION '{path}/{env}/silver/fundamentals';
    """)
    

def create_silver_prices(path):
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS hive_metastore.{env}_silver.prices (
        symbol STRING,
        current_price DOUBLE,
        open DOUBLE,
        day_high DOUBLE,
        day_low DOUBLE,
        previous_close DOUBLE,
        volume DOUBLE,
        market_cap BIGINT,
        extract_time TIMESTAMP,
        extract_date DATE,
        year INT,
        month INT,
        day INT
    )
    USING DELTA
    PARTITIONED BY (year, month, day)
    LOCATION '{path}/{env}/silver/prices';
    """)


def create_daily_aggregates(path):
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS hive_metastore.{env}_silver.daily_price_aggregates (
    symbol STRING,
    date DATE,
    open DOUBLE,
    day_high DOUBLE,
    day_low DOUBLE,
    avg_price DOUBLE,
    volume DOUBLE,
    year INT,
    month INT,
    day INT
)
USING DELTA
PARTITIONED BY (year, month, day)
LOCATION '{path}/{env}/silver/daily_price_aggregates';
    """)


def create_daily_indicators(path):
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS hive_metastore.{env}_gold.daily_price_indicators (
    symbol STRING,
    date DATE,
    open DOUBLE,
    day_high DOUBLE,
    day_low DOUBLE,
    avg_price DOUBLE,
    volume DOUBLE,
    year INT,
    month INT,
    day INT,

    -- Moving Averages
    ma_12 DOUBLE,
    ma_26 DOUBLE,
    ma_60 DOUBLE,

    -- Exponential Moving Averages
    ema_12 DOUBLE,
    ema_26 DOUBLE,
    ema_60 DOUBLE,

    -- RSI
    rsi_12 DOUBLE,
    rsi_26 DOUBLE,
    rsi_60 DOUBLE,

    -- Bollinger Bands 12
    bb_upper_12 DOUBLE,
    bb_lower_12 DOUBLE,
    bb_width_12 DOUBLE,

    -- Bollinger Bands 26
    bb_upper_26 DOUBLE,
    bb_lower_26 DOUBLE,
    bb_width_26 DOUBLE,

    -- Bollinger Bands 60
    bb_upper_60 DOUBLE,
    bb_lower_60 DOUBLE,
    bb_width_60 DOUBLE,

    -- MACD and related
    macd DOUBLE,
    signal DOUBLE,
    macd_hist DOUBLE,

    -- OBV
    obv DOUBLE
)
USING DELTA
PARTITIONED BY (year, month, day)
LOCATION '{path}/{env}/gold/daily_price_indicators';
    """)


bronze_path = 'abfss://stock-project-container@bgardzinski1stock.dfs.core.windows.net'
create_bronze_schema()
create_silver_schema()
create_gold_schema()

create_fundamental_table(bronze_path)
create_prices_table(bronze_path)

create_silver_fundamentals(bronze_path)
create_silver_prices(bronze_path)
create_daily_aggregates(bronze_path)

create_daily_indicators(bronze_path)

In [0]:
%sql DROP TABLE IF EXISTS hive_metastore.dev_silver.prices;



In [0]:
%sql DROP TABLE IF EXISTS hive_metastore.dev_silver.fundamentals;


In [0]:
%sql DROP TABLE IF EXISTS hive_metastore.dev_bronze.prices;


In [0]:
%sql DROP TABLE IF EXISTS hive_metastore.dev_bronze.fundamentals;


In [0]:
%sql DROP TABLE IF EXISTS hive_metastore.dev_silver.daily_price_aggregates;